#### Leer todos los datos que son requeridos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commont_functions"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies")\
                        .filter(f"file_date = '{v_file_date}'")

In [0]:
country_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
prod_country_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries")\
                        .filter(f"file_date = '{v_file_date}'")

In [0]:
movie_company_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies")\
                        .filter(f"file_date = '{v_file_date}'")

In [0]:
prod_company_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies")\
                        .filter(f"file_date = '{v_file_date}'")

#### Join "country" y "production_country"

In [0]:
country_prod_country_df = country_df.join(prod_country_df,
                                         country_df.country_id == prod_country_df.country_id,
                                         "inner")\
                        .select(country_df.country_name, country_df.country_id, prod_country_df.movie_id)

#### Join "production_company" y "movie_company" 

In [0]:
prod_comp_mov_comp_df = prod_company_df.join(movie_company_df,
                                         prod_company_df.company_id == movie_company_df.company_id,
                                         "inner")\
                                    .select(prod_company_df.company_name, prod_company_df.company_id, movie_company_df.movie_id)

##### Join "movies_df", "country_prod_country_df" y "prod_comp_mov_comp_df"

- Filtrar las peliculas donde su fecha de lanzamiento sea mayor o igual a 2010

In [0]:
movie_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
results_movies_country_prod_company_df = movie_filter_df.join(country_prod_country_df,
                                         movie_filter_df.movie_id == country_prod_country_df.movie_id,
                                         "inner")\
                        .join(prod_comp_mov_comp_df,
                                         movie_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,
                                         "inner")

- Agregar columna "created_date"

In [0]:
from pyspark.sql.functions import lit

In [0]:
results_df = results_movies_country_prod_company_df \
            .select(movie_filter_df.movie_id,"country_id","company_id","title","budget","revenue","duration_time","release_date","country_name","company_name")\
            .withColumn("created_date", lit(v_file_date))

- Ordenar por la columna "title" de manera descendente

In [0]:
results_order_by_df = results_df.orderBy(results_df.title.asc())

#### Escribir los datos en el DataLake en formato "delta"

In [0]:
#overwrite_partition(results_order_by_df, "movie_gold", "results_country_prod_company", "created_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date'

merge_delta_lake(results_order_by_df, "movie_gold", "results_country_prod_company", gold_folder_path, merge_condition, "created_date")




In [0]:
%sql
select * from movie_gold.results_country_prod_company

In [0]:
#display(spark.read.parquet(f"{gold_folder_path}/results_country_prod_company"))

In [0]:
#display(results_order_by_df)